In [1]:
"""Create partitions from the data model based on the Parent names

Args:
    data_model (object): dataframe that contains the data model
    partition_path (str): directory to store the partitioned data model as CSVs
"""

import glob
import os
import pandas as pd
import numpy as np
import re
import datetime
import synapseclient as sc
from pathlib import Path
from tqdm import tqdm

syn = sc.login()

dm_path = '../../EL.data.model.csv'
output_path = "../backups"

partition_path = "../modules"
# data_model = utils.load_and_backup_dm(dm_name, output_path)


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.3.0) is available. Your version (3.2.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.3.0 release notes

https://python-docs.synapse.org/news/



Welcome, Nicholas Lee!



In [2]:
dm = pd.read_csv(dm_path)
dm['module'] = dm['module'].fillna('Unknown')
dm["Parent"] = dm["Parent"].fillna("NotSet")
dm['Attribute'] = dm['Attribute'].str.strip()
dm.head()

,Attribute,Description,Valid Values,DependsOn,DependsOn Component,Required,Parent,Validation Rules,Properties,Source,module,multivalue,columnType,Ontology,Notes,UsedIn
0,acquisitionBatchID,"Acquisition batch identifier, provided by the ...",NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
1,acquisitionBatchSize,The number of samples,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
2,acquisitionBatchSizeUnit,The unit of measurement for number of samples ...,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
3,acquisitionMode,The specific aspect of a mass spectrometer met...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,https://www.ebi.ac.uk/ols/ontologies/ms/terms?...,A data contributor should be able to write in ...,Metabolomics
4,acquisitionSoftware,The name of the acquisition software used. Pro...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,http://purl.obolibrary.org/obo/MS_1001455,A data contributor should be able to write in ...,Metabolomics


In [3]:
partition_path = "../../modules"

In [21]:
# Split by module
modules = dm["module"].dropna().unique()

In [29]:
import shutil

# clean modules folders
existing_dir_mods = [s.strip('/') for s in glob.glob("**/", root_dir = partition_path)]

for e in list(set(existing_dir_mods) - set(modules)): 
    print('Removing directory: ', e)
    shutil.rmtree(Path(partition_path, e))

In [30]:
# create module directories
for m in list(set(modules) - set(existing_dir_mods)):
    mod_path = Path(partition_path, m)

    print(mod_path)

    mod_path.mkdir(parents = True, exist_ok=True)

../../modules/clinical
../../modules/Unknown
../../modules/experimentalData
../../modules/biospecimen
../../modules/batch
../../modules/ADKP
../../modules/individual
../../modules/ManifestColumn
../../modules/mass spectrometry
../../modules/template
../../modules/sequencing
../../modules/demographics


In [31]:
# # check for existing csv files
# existing_csvs = glob.glob('**/*.csv', recursive = True, root_dir = partition_path)
# existing_attrs = [Path(e).stem for e in existing_csvs]
# existing_df = pd.DataFrame(
#     {"csv_paths": existing_csvs, "existing_attrs": existing_attrs}
# )
# existing_df["module"] = existing_df["csv_paths"].apply(lambda x: str(Path(x).parent))

In [37]:
for i, r in tqdm(
    dm.iterrows(),
    desc="Updating module CSVs",
):
    try: 
        attr = r['Attribute']
        module = r['module']
        csv_path = Path(module, attr + '.csv')
        full_csv_path = Path(partition_path, csv_path)

        if not full_csv_path.exists(): 
            print('Creating new CSV: ', csv_path)
            pd.DataFrame(r).T.to_csv(full_csv_path, index=False)

        else: 
            # print('CSV already exists')
            pass

    except Exception as e: 
        print(attr, e)
        pass

Updating module CSVs: 238it [00:00, 2177.35it/s]

Creating new CSV:  ManifestColumn/acquisitionBatchID.csv
Creating new CSV:  ManifestColumn/acquisitionBatchSize.csv
Creating new CSV:  ManifestColumn/acquisitionBatchSizeUnit.csv
Creating new CSV:  ManifestColumn/acquisitionMode.csv
Creating new CSV:  ManifestColumn/acquisitionSoftware.csv
Creating new CSV:  ManifestColumn/acquisitionSoftwareVersion.csv
Creating new CSV:  ManifestColumn/adapters.csv
Creating new CSV:  ManifestColumn/adjustedCovariates.csv
Creating new CSV:  individual/age.csv
Creating new CSV:  ManifestColumn/arrayInformation.csv
Creating new CSV:  ManifestColumn/arrayManufacturer.csv
Creating new CSV:  experimentalData/assay.csv
Creating new CSV:  ManifestColumn/assemblyName.csv
Creating new CSV:  ManifestColumn/assemblyQual.csv
Creating new CSV:  ManifestColumn/assemblySoftware.csv
Creating new CSV:  ManifestColumn/associatedResource.csv
Creating new CSV:  ManifestColumn/backgroundTrait.csv
Creating new CSV:  ManifestColumn/batchID.csv
Creating new CSV:  ManifestColu